In [1]:
import numpy as np
import pandas as pd
import geopandas as gp
import matplotlib.pyplot as plt

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [2]:
pd.read_csv('../data/Raw/corona/nl_corona.csv',sep='\t')

,date,region_name,region_code,deceased_addition,confirmed_addition,hospitalized_addition,deceased_cumulative,confirmed_cumulative,hospitalized_cumulative
0,2020-02-27,Drenthe,22,NaN,NaN,NaN,NaN,0,NaN
1,2020-02-27,Flevoland,24,NaN,NaN,NaN,NaN,0,NaN
2,2020-02-27,Friesland,21,NaN,NaN,NaN,NaN,0,NaN
3,2020-02-27,Gelderland,25,NaN,NaN,NaN,NaN,0,NaN
4,2020-02-27,Groningen,20,NaN,NaN,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...
4339,2021-02-22,Noord-Holland,27,6.0,821.0,12.0,2206.0,180576,3666.0
4340,2021-02-22,Overijssel,23,1.0,197.0,2.0,1018.0,74359,1125.0
4341,2021-02-22,Utrecht,26,1.0,291.0,4.0,1111.0,84449,1971.0
4342,2021-02-22,Zeeland,29,1.0,77.0,1.0,171.0,15987,456.0
